# [1204. Last Person to Fit in the Bus](https://leetcode.com/problems/last-person-to-fit-in-the-bus/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Queue

<pre>+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| person_id   | int     |
| person_name | varchar |
| weight      | int     |
| turn        | int     |
+-------------+---------+</pre>
person_id column contains unique values.
This table has the information about all people waiting for a bus.
The person_id and turn columns will contain all numbers from 1 to n, where n is the number of rows in the table.
turn determines the order of which the people will board the bus, where turn=1 denotes the first person to board and turn=n denotes the last person to board.
weight is the weight of the person in kilograms.


There is a queue of people waiting to board a bus. However, the bus has a weight limit of 1000 kilograms, so there may be some people who cannot board.

Write a solution to find the person_name of the last person that can fit on the bus without exceeding the weight limit. The test cases are generated such that the first person does not exceed the weight limit.

The result format is in the following example.



Example 1:

Input:
Queue table:
<pre>+-----------+-------------+--------+------+
| person_id | person_name | weight | turn |
+-----------+-------------+--------+------+
| 5         | Alice       | 250    | 1    |
| 4         | Bob         | 175    | 5    |
| 3         | Alex        | 350    | 2    |
| 6         | John Cena   | 400    | 3    |
| 1         | Winston     | 500    | 6    |
| 2         | Marie       | 200    | 4    |
+-----------+-------------+--------+------+</pre>
Output:
<pre>+-------------+
| person_name |
+-------------+
| John Cena   |
+-------------+</pre>
Explanation: The folowing table is ordered by the turn for simplicity.
<pre>+------+----+-----------+--------+--------------+
| Turn | ID | Name      | Weight | Total Weight |
+------+----+-----------+--------+--------------+
| 1    | 5  | Alice     | 250    | 250          |
| 2    | 3  | Alex      | 350    | 600          |
| 3    | 6  | John Cena | 400    | 1000         | (last person to board)
| 4    | 2  | Marie     | 200    | 1200         | (cannot board)
| 5    | 4  | Bob       | 175    | ___          |
| 6    | 1  | Winston   | 500    | ___          |
+------+----+-----------+--------+--------------+</pre>

In [2]:
# pandas schema

import pandas as pd

data = [[5, 'Alice', 250, 1], [4, 'Bob', 175, 5], [3, 'Alex', 350, 2], [6, 'John Cena', 400, 3], [1, 'Winston', 500, 6],
        [2, 'Marie', 200, 4]]
queue = pd.DataFrame(data, columns=['person_id', 'person_name', 'weight', 'turn']).astype(
    {'person_id': 'Int64', 'person_name': 'object', 'weight': 'Int64', 'turn': 'Int64'})

#to spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

queue_df = spark.createDataFrame(queue)
queue_df.show(truncate=False)

+---------+-----------+------+----+
|person_id|person_name|weight|turn|
+---------+-----------+------+----+
|5        |Alice      |250   |1   |
|4        |Bob        |175   |5   |
|3        |Alex       |350   |2   |
|6        |John Cena  |400   |3   |
|1        |Winston    |500   |6   |
|2        |Marie      |200   |4   |
+---------+-----------+------+----+



In [27]:
from pyspark.sql import Window

wind= Window.orderBy(lit(1))  #max(turn) over() as m

# in Spark Dataframe
queue_df\
      .withColumn('cr_weight',sum('weight').over(Window.orderBy('turn')))\
      .withColumn('next_weight',lead('cr_weight').over(Window.orderBy('turn')))\
      .withColumn('m',max('turn').over(wind))\
      .where('(next_weight > 1000 and cr_weight <= 1000) or (turn = m and cr_weight <= 1000)')\
      .select('person_name')\
      .show()

+-----------+
|person_name|
+-----------+
|  John Cena|
+-----------+



In [23]:
# In spark sql

queue_df.createOrReplaceTempView('queue')

spark.sql('''
with cte as
    (select *,
    lead(cr_weight) over(order by turn) as next_weight
    from
    (   select *,
        sum(weight) over(order by turn) as cr_weight,
        max(turn) over() as m
        from queue)x
    )

select person_name from cte
where (next_weight > 1000 and cr_weight <= 1000) or (turn = m and cr_weight <= 1000)
''').show()

+-----------+
|person_name|
+-----------+
|  John Cena|
+-----------+



In [28]:
spark.stop()